In [1]:
# Google Sheets API Libraries
from __future__ import print_function
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import gspread
from oauth2client.service_account import ServiceAccountCredentials
 
# -----------------------------------------------------------------------#
import time

# -----------------------------------------------------------------------#

import telepot, telepot.aio
from telepot.loop import MessageLoop
from telepot.delegate import pave_event_space, per_chat_id, create_open
from telepot.loop import MessageLoop
from telepot.namedtuple import ReplyKeyboardMarkup

In [ ]:
# 3 Inputs:
# Credentials - Basically the credentials.json file
# Sheetname - The name of your spreadsheet on Google Sheets
# Range - 'A1:Z99': For you to customize, for example, you have a header row for columns, you wanna exclude that
def opener(credentials, sheetname):
    # Scope means what you want to have access to
    # First we have access to google sheets API
    # Then we have access to google drive API
    scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
 
    # Credentials is the file you downloaded
    credentials = ServiceAccountCredentials.from_json_keyfile_name(credentials, scope)

    # Authorize your credentials
    gc = gspread.authorize(credentials)

    # Opening your google sheets\n,
    # gc.open('sheetname').sheet1/2/3/4\n,
    sheet = gc.open(sheetname).sheet1
    
    # To specify the range of the final sheet produced
    # final_sheet = sheet.values_get(sheetrange)

    return sheet

In [ ]:
# Variables Store
# Google Sheets Id
sheetid = # Input your Google Sheet Id here in the format of 'sheetid'

# Bot Token
Token = # Input your Telegram Bot token here in format of 'bot token'

# Sheet Credentials
CRED = # Input the json file name of your sheet credentials here in format of'filename.json'

# Sheet Name
SNAME = # Input the name of your Google Spreadsheet here in format of 'sheetname'
# -----------------------------------------------------------------------#

# Text Libraries
welcome_text = 'Welcome to Password Manager Bot.\n\n\n\nTo add a new password entry, please type /addnew\nTo retrieve a password entry, please type /retrieve\nTo delete an existing entry, please type /delete\nTo update an existing password entry, please type /update\nTo list all password entries, please type /listall'

# /addnew text
add_new1 = 'Are you sure you want to add a new password entry? Enter /yes to confirm.'
add_new2 = 'Please enter the website that this password is used for - E.g. Hotmail'
add_new3 = 'Please enter your username.'
add_new4 = 'Please enter your password.'

# /retrieve text
r_text1 = 'Please enter the website which you which to retrieve your password for.\n\nUse /listall to view all websites that you have stored your password on.\n'

# /listall text
listall1 = 'Your Passwords are stored on the following websites:'

# /delete text
d_text1 = 'Please enter the website of the password entry which you wish to delete.\n\nUse /listall to view all websites that you have stored your password on.\n'

# /update text
u_text1 = 'Please enter the website of the password entry which you wish to update.\n\nUse /listall to view all websites that you have stored your password on.\n'

# -----------------------------------------------------------------------#
class MessageCounter(telepot.helper.ChatHandler):
    def __init__(self, *args, **kwargs):
        super(MessageCounter, self).__init__(*args, **kwargs)
        self.new_session = True
        
        # Value attributes
        self.website = ''
        self.username = ''
        self.password = ''
        
        # Stage attributes
        self.addstage = 0
        self.rstage = 0
        self.ustage = 0
        self.delstage = 0

    # Reset function     
    def reset(self):
        self.__init__()

    # Chat handler function     
    def on_chat_message(self, message):
        content_type, chat_type, chat_id = telepot.glance(message)
        
        # ----- WELCOME ----- #
        if message['text'] == '/start':
            bot.sendMessage(chat_id, welcome_text)

        
        # ----- ADD NEW ENTRY ----- # 
        # Start of process of adding new password entry into Google Sheets
        if message['text'] == '/addnew':
            self.addstage = 1
            bot.sendMessage(chat_id, add_new1)
        
        # If user inputs /yes, then will proceed to next step which is to prompt for the website
        elif message['text'] == '/yes' and self.addstage == 1:
            self.addstage = 2
            bot.sendMessage(chat_id, add_new2)
        
        # User inputs website, then bot prompts for username 
        # Previous user input is saved under the website variable
        # Move on to next step after prompting for username
        elif self.addstage == 2:
            self.website = message['text']
            bot.sendMessage(chat_id, add_new3)
            self.addstage = 3

        # User input is saved under the username variable
        # Bot prompts for the password then moves on to the next step
        elif self.addstage == 3:
            self.username = message['text']
            bot.sendMessage(chat_id, add_new4)
            self.addstage = 4
        
        # User input is saved under the password variable
        # Bot then prompts for confirmation, printing out all the previous user input
        # Move on to next step after prompt
        elif self.addstage == 4:
            self.password = message['text']
            bot.sendMessage(chat_id, 'Type /yes to confirm the following:\n' + 'Website is: ' + self.website + '\n' + 'Username is: ' + self.username + '\n' + 'Password is: ' + self.password + '\n\n' + 'If it is incorrect, type /no to start over.')
            self.addstage = 5

        # If user confirms information is correct with /yes, then bot tells user information is saved to Google Sheets
        # Open google sheets, then add a new row containing all the information previously input by user
        elif message['text'] == '/yes' and self.addstage == 5:
            bot.sendMessage(chat_id, 'Ok, your entry has been saved onto Google Sheets for future reference.')
            sht = opener(CRED, SNAME)
            sht.append_row([self.website, self.username, self.password], value_input_option='RAW')
            self.addstage = 0
            self.reset

        # If user answers with /no, bot wil prompt user to start over
        elif message['text'] == '/no' and self.addstage == 5:
            bot.sendMessage(chat_id, 'Please type /addnew to start over.')
            self.addstage = 0
            self.reset
            
            
        # ----- RETRIEVE PASSWORD ----- #
        # If user inputs /retrieve bot will send prompt asking user what website's password user wishes to retrieve
        # Move on to rstage 1
        if message['text'] == '/retrieve':
            bot.sendMessage(chat_id, r_text1)
            self.rstage = 1
        
        # Saves user input as self.website
        # Uses r_web as a search value for the Google Sheets using gspread
        # .find() will produce a gpsread cell object, which has .row that can tell you which row the value is located on
        # Then use .row_values(cell.row)[N] to extract out the website, username and password from Google Sheets
        # Bot will these information back to the user, then self.rstage is resetted back to 0
        elif self.rstage == 1:
            self.website  = message['text']
            sht = opener(CRED, SNAME)
            cell = sht.find(self.website)
            web = sht.row_values(cell.row)[0]
            user = sht.row_values(cell.row)[1]
            pword = sht.row_values(cell.row)[2]
            reply = 'For ' + web + ':\n' + 'Username is: ' + user +'\n' + 'Password is: ' + pword
            bot.sendMessage(chat_id, reply)
            self.rstage = 0
            self.reset
        
        
        # ----- DELETE AN ENTRY ----- #
        # If user inputs /delete, bot will start the process for deleting an entry
        # Bot will ask user which website they wish to delete it from
        # self.delstage = 1 to proceed to next step
        if message['text'] == '/delete':
            bot.sendMessage(chat_id, d_text1)
            self.delstage = 1
            
        # User input is saved as self.website which will be used to find the row that contains this website
        # Then entire row is deleted with .delete_row, bot then sends a message telling user that the entry has been deleted
        # self.delstage resets to 0
        elif self.delstage == 1:
            self.website = message['text']
            sht = opener(CRED, SNAME)
            cell = sht.find(self.website)
            sht.delete_row(cell.row)
            reply = 'Your entry for ' + self.website + ' has been deleted from Google Sheets.' 
            bot.sendMessage(chat_id, reply)
            self.delstage = 0
            self.reset
        
        
        # ----- UPDATE AN ENTRY ----- #
        # If user inputs /update, bot will prompt user asking for the website of entry that they wanna update
        # self.ustage = 1 to move on to next step
        if message['text'] == '/update':
            bot.sendMessage(chat_id, u_text1)
            self.ustage = 1
            
        # User input website is saved as u_site
        # Bot prompts for updated username
        # Move on to next step with self.ustage = 2
        elif self.ustage == 1:
            self.website = message['text']
            bot.sendMessage(chat_id, 'Please enter your updated username.')
            self.ustage = 2
            
        # User input username is saved as u_user
        # Bot prompts for updated password
        # Move on to next step with self.ustage = 3
        
        elif self.ustage == 2:
            self.username = message['text']
            bot.sendMessage(chat_id, 'Please enter your updated password.')
            self.ustage = 3
        # User input password is saved as u_pass
        # Bot prompts for user to confirm that the previously entered information is correct
        # Move on to next step with self.ustage = 4
        
        elif self.ustage == 3:
            self.password = message['text']
            bot.sendMessage(chat_id, 'Type /yes to confirm the following:\n' + 'Updated username is: ' + self.username + '\n' + 'Updated password is: ' + self.password + '\n\n' + 'If it is incorrect, type /no to start over.')
            self.ustage = 4
            
        # If user inputs /yes, then will use user entered website to find the row that contains that information with sht.find()
        # Then will update the username & password with .update_cell()
        # Bot then sends message to user that information has been updated
        # self.ustage resetted to 0
        elif message['text'] == '/yes' and self.ustage == 4:
            sht = opener(CRED, SNAME)
            cell = sht.find(self.website)
            # Update the username 
            sht.update_cell(cell.row, cell.col+1, self.username)
            # Update the password
            sht.update_cell(cell.row, cell.col+2, self.password)
            reply = 'Ok, your username and password for ' + self.website + ' has been updated.'
            bot.sendMessage(chat_id, reply)
            self.ustage = 0
            self.reset
            
        # If user inputs /no, then bot will prompt user to restart over
        # self.ustage resetted to 0
        elif message['text'] == '/no' and self.ustage == 4:
            bot.sendMessage(chat_id, 'Please type /update to start over.')
            self.ustage
            self.reset
        
        # ----- LIST ALL ENTRIES ----- #
        # If user inputs /listall, then bot will send a list of all the websites user stored their password on
        # Use gspread to pull the data using .get_all_records(), then store in a list called sites, then bot send out sites
        if message['text'] == '/listall':
            sht = opener(CRED, SNAME)
            bot.sendMessage(chat_id, listall1)
            sites= []
            for values in sht.get_all_records():
                sites.append([values['Website']])
            bot.sendMessage(chat_id, sites)

            
max_duration = 300



bot = telepot.DelegatorBot(Token, [
    pave_event_space()(
        per_chat_id(), create_open, MessageCounter, timeout=max_duration),
])



run_server = True
print('BOT IS UP! WOOT WOOT!.')
MessageLoop(bot).run_as_thread()

                  

while run_server:
    time.sleep(3)